In [21]:
import pandas as pd
import numpy as np

In [2]:
cda = pd.read_csv('ontario_da.csv')

/Users/Rick/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
cda = cda[cda['GEO_LEVEL'] == 4]

In [14]:
cda['subdivision'] = (cda['GEO_CODE (POR)']/10000).astype(int)

In [16]:
cda = cda[cda['subdivision'] == 3520]

In [38]:
cda['Dim: Sex (3): Member ID: [1]: Total - Sex'] = pd.to_numeric(cda['Dim: Sex (3): Member ID: [1]: Total - Sex'], errors='coerce')
cda['Dim: Sex (3): Member ID: [2]: Male'] = pd.to_numeric(cda['Dim: Sex (3): Member ID: [2]: Male'], errors='coerce')
cda['Dim: Sex (3): Member ID: [3]: Female'] = pd.to_numeric(cda['Dim: Sex (3): Member ID: [3]: Female'], errors='coerce')

/Users/Rick/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/Rick/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Rick/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [42]:
cda = cda.reset_index()
cda = cda.rename(columns = {'GEO_CODE (POR)':'DAUID','DIM: Profile of Dissemination Areas (2247)':'profile',
                           'Member ID: Profile of Dissemination Areas (2247)':'member_id',
                           'Dim: Sex (3): Member ID: [1]: Total - Sex':'pop',
                           'Dim: Sex (3): Member ID: [2]: Male':'male',
                           'Dim: Sex (3): Member ID: [3]: Female':'female'})
cda_simple = cda[['DAUID', 'profile','member_id', 'pop', 'male', 'female']].copy()

In [45]:
del(cda)

In [48]:
cda_simple

,DAUID,profile,member_id,pop,male,female
0,35200002,"Population, 2016",1,356.0,NaN,NaN
1,35200002,"Population, 2011",2,NaN,NaN,NaN
2,35200002,"Population percentage change, 2011 to 2016",3,NaN,NaN,NaN
3,35200002,Total private dwellings,4,108.0,NaN,NaN
4,35200002,Private dwellings occupied by usual residents,5,100.0,NaN,NaN
...,...,...,...,...,...,...
8318389,35204906,Migrants,2243,430.0,225.0,200.0
8318390,35204906,Internal migrants,2244,85.0,40.0,40.0
8318391,35204906,Intraprovincial migrants,2245,55.0,35.0,20.0
8318392,35204906,Interprovincial migrants,2246,30.0,10.0,15.0


In [79]:
# profile "Population" doesn't split between male and female

cda_pop = cda_simple[cda_simple['member_id'] == 9].append(
    cda_simple[cda_simple['member_id'] == 13]).append(
    cda_simple[cda_simple['member_id'] == 24])

In [80]:
cda_pop = cda_pop.groupby('DAUID').sum()[['male', 'female']].reset_index()
cda_pop['pop'] = cda_pop['male'] + cda_pop['female']
cda_pop['profile'] = 'Population, 2016'
cda_pop['member_id'] = 1
cda_pop = cda_pop[['DAUID', 'male', 'female']]
cda_pop

,DAUID,male,female
0,35200002,185.0,175.0
1,35200003,165.0,170.0
2,35200004,180.0,200.0
3,35200005,195.0,210.0
4,35200006,265.0,265.0
...,...,...,...
3697,35204902,230.0,260.0
3698,35204903,1730.0,1765.0
3699,35204904,315.0,315.0
3700,35204905,870.0,910.0


In [82]:
cda_lep = cda_simple[cda_simple['member_id'] == 104]
cda_lep = cda_lep.rename(columns = {'male':'male_lep', 'female':'female_lep'})
cda_pop = cda_pop.merge(cda_lep[['DAUID', 'male_lep', 'female_lep']])
cda_pop

,DAUID,male,female,male_lep,female_lep
0,35200002,185.0,175.0,5.0,10.0
1,35200003,165.0,170.0,5.0,15.0
2,35200004,180.0,200.0,5.0,10.0
3,35200005,195.0,210.0,5.0,10.0
4,35200006,265.0,265.0,10.0,15.0
...,...,...,...,...,...
3697,35204902,230.0,260.0,5.0,5.0
3698,35204903,1730.0,1765.0,85.0,115.0
3699,35204904,315.0,315.0,5.0,10.0
3700,35204905,870.0,910.0,45.0,60.0


## Knowledge of Official Languages

In [65]:
cda_lep = cda_simple[cda_simple['member_id'] == 104]
cda_lep = cda_lep.rename(columns = {'male':'male_lep', 'female':'female_lep'})
cda_pop = cda_pop.merge(cda_lep[['DAUID', 'male_lep', 'female_lep']])
cda_pop

,DAUID,profile,member_id,pop,male,female
103,35200002,Neither English nor French,104,15.0,5.0,10.0
2350,35200003,Neither English nor French,104,20.0,5.0,15.0
4597,35200004,Neither English nor French,104,15.0,5.0,10.0
6844,35200005,Neither English nor French,104,15.0,5.0,10.0
9091,35200006,Neither English nor French,104,25.0,10.0,15.0
...,...,...,...,...,...,...
8307262,35204902,Neither English nor French,104,10.0,5.0,5.0
8309509,35204903,Neither English nor French,104,205.0,85.0,115.0
8311756,35204904,Neither English nor French,104,10.0,5.0,10.0
8314003,35204905,Neither English nor French,104,105.0,45.0,60.0


# 25% sample data

In [83]:
cda_25 = cda_simple[cda_simple['member_id'] == 1323]
cda_25 = cda_25[['DAUID', 'male', 'female']]

## Total Visible Minority (excluding Aboriginal)

In [90]:
cda_rac = cda_simple[cda_simple['member_id'] == 1324]

cda_rac = cda_rac.rename(columns = {'male':'male_rac', 'female':'female_rac'})
cda_25 = cda_25.merge(cda_rac[['DAUID', 'male_rac', 'female_rac']])
cda_25

,DAUID,male,female,male_rac,female_rac
0,35200002,230.0,160.0,215.0,155.0
1,35200003,150.0,125.0,150.0,130.0
2,35200004,190.0,235.0,185.0,235.0
3,35200005,220.0,280.0,225.0,280.0
4,35200006,210.0,210.0,200.0,200.0
...,...,...,...,...,...
3697,35204902,235.0,300.0,135.0,185.0
3698,35204903,1750.0,1795.0,1425.0,1540.0
3699,35204904,280.0,350.0,125.0,230.0
3700,35204905,925.0,895.0,695.0,715.0


## Black

In [91]:
cda_bl = cda_simple[cda_simple['member_id'] == 1327]

cda_bl = cda_bl.rename(columns = {'male':'male_bl', 'female':'female_bl'})
cda_25 = cda_25.merge(cda_bl[['DAUID', 'male_bl', 'female_bl']])
cda_25

,DAUID,male,female,male_rac,female_rac,male_bl,female_bl
0,35200002,230.0,160.0,215.0,155.0,45.0,40.0
1,35200003,150.0,125.0,150.0,130.0,25.0,20.0
2,35200004,190.0,235.0,185.0,235.0,20.0,20.0
3,35200005,220.0,280.0,225.0,280.0,15.0,30.0
4,35200006,210.0,210.0,200.0,200.0,25.0,45.0
...,...,...,...,...,...,...,...
3697,35204902,235.0,300.0,135.0,185.0,10.0,0.0
3698,35204903,1750.0,1795.0,1425.0,1540.0,65.0,60.0
3699,35204904,280.0,350.0,125.0,230.0,10.0,20.0
3700,35204905,925.0,895.0,695.0,715.0,35.0,25.0


## Aboriginal

In [94]:
cda_ab = cda_simple[cda_simple['member_id'] == 1302]

cda_ab = cda_ab.rename(columns = {'male':'male_ab', 'female':'female_ab'})
cda_25 = cda_25.merge(cda_ab[['DAUID', 'male_ab', 'female_ab']])
cda_25

,DAUID,male,female,male_rac,female_rac,male_bl,female_bl,male_ab,female_ab
0,35200002,230.0,160.0,215.0,155.0,45.0,40.0,0.0,0.0
1,35200003,150.0,125.0,150.0,130.0,25.0,20.0,0.0,0.0
2,35200004,190.0,235.0,185.0,235.0,20.0,20.0,0.0,0.0
3,35200005,220.0,280.0,225.0,280.0,15.0,30.0,0.0,0.0
4,35200006,210.0,210.0,200.0,200.0,25.0,45.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3697,35204902,235.0,300.0,135.0,185.0,10.0,0.0,0.0,0.0
3698,35204903,1750.0,1795.0,1425.0,1540.0,65.0,60.0,0.0,0.0
3699,35204904,280.0,350.0,125.0,230.0,10.0,20.0,0.0,0.0
3700,35204905,925.0,895.0,695.0,715.0,35.0,25.0,0.0,0.0


In [100]:
cda_25['male_rac'] = cda_25['male_rac'] + cda_25['male_ab']
cda_25['female_rac'] = cda_25['female_rac'] + cda_25['female_ab']

## Immigrated 2011-2016

In [99]:
cda_imm = cda_simple[cda_simple['member_id'] == 1149]

cda_imm = cda_imm.rename(columns = {'male':'male_imm', 'female':'female_imm'})
cda_25 = cda_25.merge(cda_imm[['DAUID', 'male_imm', 'female_imm']])
cda_25

,DAUID,male,female,male_rac,female_rac,male_bl,female_bl,male_ab,female_ab,male_imm,female_imm
0,35200002,230.0,160.0,215.0,155.0,45.0,40.0,0.0,0.0,15.0,0.0
1,35200003,150.0,125.0,150.0,130.0,25.0,20.0,0.0,0.0,0.0,10.0
2,35200004,190.0,235.0,185.0,235.0,20.0,20.0,0.0,0.0,0.0,10.0
3,35200005,220.0,280.0,225.0,280.0,15.0,30.0,0.0,0.0,10.0,0.0
4,35200006,210.0,210.0,200.0,200.0,25.0,45.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3697,35204902,235.0,300.0,135.0,185.0,10.0,0.0,0.0,0.0,10.0,10.0
3698,35204903,1750.0,1795.0,1425.0,1540.0,65.0,60.0,0.0,0.0,330.0,385.0
3699,35204904,280.0,350.0,125.0,230.0,10.0,20.0,0.0,0.0,20.0,55.0
3700,35204905,925.0,895.0,695.0,715.0,35.0,25.0,0.0,0.0,185.0,185.0


In [101]:
cda_25.columns

Index(['DAUID', 'male', 'female', 'male_rac', 'female_rac', 'male_bl',
       'female_bl', 'male_ab', 'female_ab', 'male_imm', 'female_imm'],
      dtype='object')

In [102]:
cda_pop

,DAUID,male,female,male_lep,female_lep
0,35200002,185.0,175.0,5.0,10.0
1,35200003,165.0,170.0,5.0,15.0
2,35200004,180.0,200.0,5.0,10.0
3,35200005,195.0,210.0,5.0,10.0
4,35200006,265.0,265.0,10.0,15.0
...,...,...,...,...,...
3697,35204902,230.0,260.0,5.0,5.0
3698,35204903,1730.0,1765.0,85.0,115.0
3699,35204904,315.0,315.0,5.0,10.0
3700,35204905,870.0,910.0,45.0,60.0


In [103]:
cda_25.to_csv('cda_25.csv', index = False)
cda_pop.to_csv('cda_pop.csv', index = False)